## Preprocessing

In [31]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np

In [32]:
X = np.fromfile('X_data.csv', dtype=float, sep=',')
y = np.fromfile('y_data.csv', dtype=np.int8, sep=',')
X = X.reshape((8400, 11, 29))
y = y.reshape((8400, 2))
X.shape, y.shape

((8400, 11, 29), (8400, 2))

In [33]:
counts = [0, 0, 0, 0]
for result in y:
    if np.array_equal(result, [0, 0]):
        counts[0] += 1
    elif np.array_equal(result, [0, 1]):
        counts[1] += 1
    elif np.array_equal(result, [1, 0]):
        counts[2] += 1
    else:
        counts[3] += 1
counts

[5156, 1506, 1738, 0]

In [34]:
X_fl = X.reshape(8400, 11*29)
X_fl.shape

(8400, 319)

In [48]:
y_cat = []
for sample in y:
    if np.array_equal(sample, [0, 0]):
        y_cat.append([0, 0, 0])
    elif np.array_equal(sample, [1, 0]):
        y_cat.append([0, 1, 0])
    else:
        y_cat.append([0, 0, 1])
y_cat = np.array(y_cat, dtype=np.int8)

In [49]:
counts = [0, 0, 0, 0]
for result in y_cat:
    if np.array_equal(result, [0, 0, 0]):
        counts[0] += 1
    elif np.array_equal(result, [0, 0, 1]):
        counts[1] += 1
    elif np.array_equal(result, [0, 1, 0]):
        counts[2] += 1
    else:
        counts[3] += 1
counts


[5156, 1506, 1738, 0]

## Simpe NN

In [50]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [72]:
model = Sequential([
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy',])

In [73]:
history = model.fit(X_fl, y_cat, epochs=500, batch_size=32, verbose=2)

Epoch 1/500
263/263 - 1s - loss: 0.2491 - categorical_accuracy: 0.3133
Epoch 2/500
263/263 - 1s - loss: 0.1779 - categorical_accuracy: 0.3432
Epoch 3/500
263/263 - 0s - loss: 0.1617 - categorical_accuracy: 0.3496
Epoch 4/500
263/263 - 0s - loss: 0.1527 - categorical_accuracy: 0.3546
Epoch 5/500
263/263 - 0s - loss: 0.1479 - categorical_accuracy: 0.3544
Epoch 6/500
263/263 - 0s - loss: 0.1423 - categorical_accuracy: 0.3574
Epoch 7/500
263/263 - 1s - loss: 0.1355 - categorical_accuracy: 0.3593
Epoch 8/500
263/263 - 1s - loss: 0.1330 - categorical_accuracy: 0.3581
Epoch 9/500
263/263 - 1s - loss: 0.1283 - categorical_accuracy: 0.3620
Epoch 10/500
263/263 - 1s - loss: 0.1246 - categorical_accuracy: 0.3624
Epoch 11/500
263/263 - 0s - loss: 0.1199 - categorical_accuracy: 0.3642
Epoch 12/500
263/263 - 1s - loss: 0.1155 - categorical_accuracy: 0.3629
Epoch 13/500
263/263 - 1s - loss: 0.1149 - categorical_accuracy: 0.3645
Epoch 14/500
263/263 - 1s - loss: 0.1097 - categorical_accuracy: 0.3656
E

Simple NN has low quality

## LSTM

In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.25, random_state=42)

In [80]:
from keras.layers import LSTM

In [94]:
model = Sequential()
model.add(LSTM(64, input_shape = X[0].shape, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32, input_shape = X[0].shape))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer="adam")

In [95]:
model.fit(X_train, y_train, batch_size=64, epochs=200, verbose=2)

Epoch 1/200
99/99 - 3s - loss: 0.3819 - binary_accuracy: 0.7910
Epoch 2/200
99/99 - 1s - loss: 0.2238 - binary_accuracy: 0.7745
Epoch 3/200
99/99 - 1s - loss: 0.1884 - binary_accuracy: 0.7803
Epoch 4/200
99/99 - 1s - loss: 0.1740 - binary_accuracy: 0.7815
Epoch 5/200
99/99 - 1s - loss: 0.1643 - binary_accuracy: 0.7813
Epoch 6/200
99/99 - 1s - loss: 0.1544 - binary_accuracy: 0.7785
Epoch 7/200
99/99 - 1s - loss: 0.1473 - binary_accuracy: 0.7793
Epoch 8/200
99/99 - 1s - loss: 0.1394 - binary_accuracy: 0.7812
Epoch 9/200
99/99 - 1s - loss: 0.1388 - binary_accuracy: 0.7803
Epoch 10/200
99/99 - 1s - loss: 0.1310 - binary_accuracy: 0.7812
Epoch 11/200
99/99 - 1s - loss: 0.1277 - binary_accuracy: 0.7814
Epoch 12/200
99/99 - 1s - loss: 0.1284 - binary_accuracy: 0.7807
Epoch 13/200
99/99 - 1s - loss: 0.1222 - binary_accuracy: 0.7835
Epoch 14/200
99/99 - 1s - loss: 0.1251 - binary_accuracy: 0.7811
Epoch 15/200
99/99 - 1s - loss: 0.1170 - binary_accuracy: 0.7843
Epoch 16/200
99/99 - 1s - loss: 0.

In [93]:
model.evaluate(X_test, y_test)

66/66 [==============================] - 1s 3ms/step - loss: 0.1145 - binary_accuracy: 0.7951


[0.11447743326425552, 0.795079231262207]

## Convolutional NN

In [180]:
from keras.layers import MaxPooling1D, Flatten, Conv1D

# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(11, 29)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy',])

In [181]:
history = model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
197/197 - 1s - loss: 0.9672 - binary_accuracy: 0.8808
Epoch 2/100
197/197 - 0s - loss: 1.1043 - binary_accuracy: 0.8861
Epoch 3/100
197/197 - 0s - loss: 0.7851 - binary_accuracy: 0.9140
Epoch 4/100
197/197 - 0s - loss: 0.6462 - binary_accuracy: 0.9238
Epoch 5/100
197/197 - 0s - loss: 0.7071 - binary_accuracy: 0.9263
Epoch 6/100
197/197 - 0s - loss: 0.6625 - binary_accuracy: 0.9090
Epoch 7/100
197/197 - 0s - loss: 0.4249 - binary_accuracy: 0.9298
Epoch 8/100
197/197 - 0s - loss: 1.2784 - binary_accuracy: 0.8756
Epoch 9/100
197/197 - 0s - loss: 0.4539 - binary_accuracy: 0.9362
Epoch 10/100
197/197 - 0s - loss: 0.4022 - binary_accuracy: 0.9375
Epoch 11/100
197/197 - 0s - loss: 0.4224 - binary_accuracy: 0.9344
Epoch 12/100
197/197 - 0s - loss: 0.3958 - binary_accuracy: 0.9399
Epoch 13/100
197/197 - 0s - loss: 0.3056 - binary_accuracy: 0.9440
Epoch 14/100
197/197 - 0s - loss: 0.3269 - binary_accuracy: 0.9434
Epoch 15/100
197/197 - 0s - loss: 0.5718 - binary_accuracy: 0.9214
Epoc

In [182]:
model.evaluate(X_test, y_test)

66/66 [==============================] - 0s 908us/step - loss: 0.6057 - binary_accuracy: 0.9448


[0.6056530475616455, 0.9447619915008545]

In [183]:
from keras.layers import BatchNormalization, ReLU

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, padding='same', input_shape=(11, 29)))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv1D(filters=64, kernel_size=2, padding='same', input_shape=(11, 29)))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv1D(filters=64, kernel_size=2, padding='same', input_shape=(11, 29)))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy',])

In [184]:
history = model.fit(X_train, y_train, epochs=500, verbose=2)

Epoch 1/500
197/197 - 1s - loss: 1.6770 - binary_accuracy: 0.8678
Epoch 2/500
197/197 - 1s - loss: 1.4887 - binary_accuracy: 0.8843
Epoch 3/500
197/197 - 1s - loss: 1.0731 - binary_accuracy: 0.8994
Epoch 4/500
197/197 - 1s - loss: 0.7553 - binary_accuracy: 0.9107
Epoch 5/500
197/197 - 1s - loss: 0.6967 - binary_accuracy: 0.9090
Epoch 6/500
197/197 - 1s - loss: 1.0386 - binary_accuracy: 0.8784
Epoch 7/500
197/197 - 1s - loss: 0.7778 - binary_accuracy: 0.8962
Epoch 8/500
197/197 - 1s - loss: 0.7049 - binary_accuracy: 0.9009
Epoch 9/500
197/197 - 1s - loss: 0.4221 - binary_accuracy: 0.9237
Epoch 10/500
197/197 - 1s - loss: 0.4004 - binary_accuracy: 0.9226
Epoch 11/500
197/197 - 1s - loss: 0.4577 - binary_accuracy: 0.9147
Epoch 12/500
197/197 - 1s - loss: 0.2510 - binary_accuracy: 0.9245
Epoch 13/500
197/197 - 1s - loss: 0.2315 - binary_accuracy: 0.9259
Epoch 14/500
197/197 - 1s - loss: 0.2051 - binary_accuracy: 0.9326
Epoch 15/500
197/197 - 1s - loss: 0.1909 - binary_accuracy: 0.9349
Epoc

In [185]:
model.evaluate(X_test, y_test)

66/66 [==============================] - 0s 1ms/step - loss: 0.2225 - binary_accuracy: 0.9675


[0.22249718010425568, 0.9674603343009949]